In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [19]:
df = pd.read_csv("/content/movies.csv")

In [8]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [7]:
df.shape

(4803, 24)

In [10]:
df['crew']

,crew
0,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd..."
1,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa..."
2,"[{'name': 'Thomas Newman', 'gender': 2, 'depar..."
3,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm..."
4,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa..."
...,...
4798,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de..."
4799,"[{'name': 'Edward Burns', 'gender': 2, 'depart..."
4800,"[{'name': 'Carla Hetland', 'gender': 0, 'depar..."
4801,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm..."


In [8]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [20]:
df2 = df.drop(
    labels=['budget', 'homepage', 'id', 'revenue', 'title', 'crew',
            'original_language', 'original_title', 'production_countries',
            'release_date', 'runtime', 'spoken_languages'],
    axis=1
)

In [21]:
df2

,index,genres,keywords,overview,popularity,production_companies,status,tagline,vote_average,vote_count,cast,director
0,0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Released,Enter the World of Pandora.,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",Released,"At the end of the world, the adventure begins.",6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",Released,A Plan No One Escapes,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",Released,The Legend Ends,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",Released,"Lost in our world, found in another.",6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]",Released,"He didn't come looking for trouble, but troubl...",6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,4799,Comedy Romance,NaN,A newlywed couple's honeymoon is upended by th...,0.642552,[],Released,A newlywed couple's honeymoon is upended by th...,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...",Released,NaN,7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,4801,NaN,NaN,When ambitious New York attorney Sam is sent t...,0.857008,[],Released,A New Yorker in Shanghai,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [23]:
df2.shape

(4803, 12)

In [24]:
df2.isnull().sum()


,0
index,0
genres,28
keywords,412
overview,3
popularity,0
production_companies,0
status,0
tagline,844
vote_average,0
vote_count,0


In [25]:
df2.fillna('', inplace = True)

In [23]:
df.head()

,genres,keywords,overview,popularity,production_companies,status,tagline,vote_average,vote_count,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Released,Enter the World of Pandora.,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",Released,"At the end of the world, the adventure begins.",6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",Released,A Plan No One Escapes,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",Released,The Legend Ends,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",Released,"Lost in our world, found in another.",6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [26]:
df.popularity

,popularity
0,150.437577
1,139.082615
2,107.376788
3,112.312950
4,43.926995
...,...
4798,14.269792
4799,0.642552
4800,1.444476
4801,0.857008


In [26]:
df2.columns

Index(['index', 'genres', 'keywords', 'overview', 'popularity',
       'production_companies', 'status', 'tagline', 'vote_average',
       'vote_count', 'cast', 'director'],
      dtype='object')

In [30]:
combined_feature = df2['genres'] + ' ' + df2['keywords'] + ' ' + df2['overview'] + ' ' + df2['tagline'] + ' ' + df2['cast'] + ' ' + df2['director'] + ' ' + df2['production_companies']

In [31]:
combined_feature

,0
0,Action Adventure Fantasy Science Fiction cultu...
1,Adventure Fantasy Action ocean drug abuse exot...
2,Action Adventure Crime spy based on novel secr...
3,Action Crime Drama Thriller dc comics crime fi...
4,Action Adventure Science Fiction based on nove...
...,...
4798,Action Crime Thriller united states\u2013mexic...
4799,Comedy Romance A newlywed couple's honeymoon ...
4800,Comedy Drama Romance TV Movie date love at fir...
4801,When ambitious New York attorney Sam is sent...


In [28]:
vectorizer = TfidfVectorizer()

In [31]:
feature_vector = vectorizer.fit_transform(combined_feature)

In [35]:
print(feature_vector)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 360164 stored elements and shape (4803, 38087)>
  Coords	Values
  (0, 5543)	0.05367124936545135
  (0, 5690)	0.06154297339087535
  (0, 15441)	0.07654459877197602
  (0, 30845)	0.07124742404505298
  (0, 15728)	0.0715341210954062
  (0, 12090)	0.13050571897807273
  (0, 10688)	0.13936740313557971
  (0, 16610)	0.09910933518936761
  (0, 32393)	0.2183604882350377
  (0, 36858)	0.07770649558575558
  (0, 11022)	0.1541753984741682
  (0, 32190)	0.11921505336163431
  (0, 19505)	0.032789352785015984
  (0, 34236)	0.07457189605911528
  (0, 1928)	0.1773895802283129
  (0, 10103)	0.16795385206514835
  (0, 26706)	0.17327889069926256
  (0, 23319)	0.1288751504840502
  (0, 20086)	0.03747200431363003
  (0, 13371)	0.16176162647823283
  (0, 34545)	0.027885782023034364
  (0, 24662)	0.13107654367927438
  (0, 26633)	0.29701834008476224
  (0, 26090)	0.04414701774017536
  (0, 35908)	0.13854744472408814
  :	:
  (4802, 22731)	0.0925576821264418
  (4802, 11589

In [32]:
similarity = cosine_similarity(feature_vector)

In [16]:
similarity.shape

(4803, 4803)

In [17]:
similarity

array([[1.        , 0.05596506, 0.04124366, ..., 0.03534218, 0.0276738 ,
        0.0164769 ],
       [0.05596506, 1.        , 0.04880217, ..., 0.05138505, 0.02877342,
        0.02877794],
       [0.04124366, 0.04880217, 1.        , ..., 0.03009804, 0.04432021,
        0.01685446],
       ...,
       [0.03534218, 0.05138505, 0.03009804, ..., 1.        , 0.04074648,
        0.03655312],
       [0.0276738 , 0.02877342, 0.04432021, ..., 0.04074648, 1.        ,
        0.02946268],
       [0.0164769 , 0.02877794, 0.01685446, ..., 0.03655312, 0.02946268,
        1.        ]])

In [33]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [34]:
list_of_movies = df['title'].to_list()

In [35]:
movie_name = input("enter your desire movie name")

enter your desire movie nameiton man


In [36]:
find_close_match = difflib.get_close_matches(movie_name, list_of_movies)
find_close_match

['Hitman', 'Rain Man', 'Iron Man']

In [37]:
close_match = find_close_match[0]

In [42]:
index_of_movie = df[df.title == close_match]['index'].values[0]
index_of_movie

np.int64(2398)

In [43]:
similarity_score = list(enumerate(similarity[index_of_movie]))
similarity_score

[(0, np.float64(0.09840861022241063)),
 (1, np.float64(0.049736342098292086)),
 (2, np.float64(0.06259448339514727)),
 (3, np.float64(0.05381597351919485)),
 (4, np.float64(0.02673453106239825)),
 (5, np.float64(0.038840706771876995)),
 (6, np.float64(0.025527230949212946)),
 (7, np.float64(0.051760519182461946)),
 (8, np.float64(0.024193017085605242)),
 (9, np.float64(0.054463332660361095)),
 (10, np.float64(0.042977746270397174)),
 (11, np.float64(0.09411532435684607)),
 (12, np.float64(0.02998862820669097)),
 (13, np.float64(0.03659757421299407)),
 (14, np.float64(0.05367759709061114)),
 (15, np.float64(0.03845179921432931)),
 (16, np.float64(0.05534330428142741)),
 (17, np.float64(0.06098003014494404)),
 (18, np.float64(0.07185172011155365)),
 (19, np.float64(0.03155320683459919)),
 (20, np.float64(0.04349474280613215)),
 (21, np.float64(0.061948052679351495)),
 (22, np.float64(0.023782890480959207)),
 (23, np.float64(0.04086547675995718)),
 (24, np.float64(0.046855175571927206)),


In [44]:
sorted_similer_movies = sorted(similarity_score, key = lambda x:x[1], reverse=True)
sorted_similer_movies

[(2398, np.float64(1.0)),
 (261, np.float64(0.15710420118801616)),
 (1398, np.float64(0.1516279059540116)),
 (2646, np.float64(0.14095440985811095)),
 (668, np.float64(0.14074853451847227)),
 (753, np.float64(0.13578932336826857)),
 (1245, np.float64(0.13221801703137975)),
 (1020, np.float64(0.13006100538980594)),
 (1513, np.float64(0.12283867518338343)),
 (1096, np.float64(0.12170169960404745)),
 (213, np.float64(0.12043724580580151)),
 (101, np.float64(0.11916454939680224)),
 (1815, np.float64(0.11476514435236077)),
 (335, np.float64(0.1144484470672813)),
 (122, np.float64(0.11360656094263295)),
 (84, np.float64(0.11218888199002996)),
 (33, np.float64(0.11210218332683385)),
 (1118, np.float64(0.11126379126958147)),
 (939, np.float64(0.11107399649852512)),
 (215, np.float64(0.11017166584834777)),
 (778, np.float64(0.10922685370012296)),
 (1652, np.float64(0.1091452927804834)),
 (220, np.float64(0.10872776725517297)),
 (1013, np.float64(0.10807747675920117)),
 (843, np.float64(0.106129

In [45]:
df[df['index'] == 261]

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
261,261,110000000,Action Thriller,NaN,1571,usa washington d.c. helicopter hostage fbi,en,Live Free or Die Hard,"John McClane is back and badder than ever, and...",48.93337,...,128.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The old school cop is back!,Live Free or Die Hard,6.4,2089,Bruce Willis Justin Long Timothy Olyphant Clif...,"[{'name': 'William Wisher Jr.', 'gender': 2, '...",Len Wiseman
